### Génération de matrice aléatoires

In [1]:
import time

# Méthode 1 : Remplir la matrice aléatoirement puis ajuster les zéros et la diagonale
def generate_triangular_method1(n, l):
    U1 = MatrixSpace(ZZ, n, n).random_element(x=-(l*10), y=(l*10) + 1, distribution='uniform')
    U2 = MatrixSpace(ZZ, n, n).random_element(x=-(l*10), y=(l*10) + 1, distribution='uniform')
    
    for i in range(n):
        U1[i, i], U2[i, i] = choice([-1, 1]), choice([-1, 1])
        for j in range(i):
            U1[i, j], U2[i, j] = 0, 0
    return U1, U2

# Méthode 2 : Générer directement des valeurs au bon endroit pour une matrice triangulaire
def generate_triangular_method2(n, l):
    U1 = MatrixSpace(ZZ, n, n)(0)
    U2 = MatrixSpace(ZZ, n, n)(0)
    
    for i in range(n):
        U1[i, i], U2[i, i] = choice([-1, 1]), choice([-1, 1])
        for j in range(i + 1, n):
            U1[j, i], U2[j, i] = randint(-(l*10), (l*10)), randint(-(l*10), (l*10))
    return U1, U2

def compare_triangular_generation(n, l, repetitions=100):
    
    start_time = time.time()
    for _ in range(repetitions):
        generate_triangular_method1(n, l)
    time_method1 = (time.time() - start_time) / repetitions

    start_time = time.time()
    for _ in range(repetitions):
        generate_triangular_method2(n, l)
    time_method2 = (time.time() - start_time) / repetitions

    print(f"Temps moyen pour la Méthode 1 : {time_method1:.6f} secondes")
    print(f"Temps moyen pour la Méthode 2 : {time_method2:.6f} secondes")

In [2]:
compare_triangular_generation(n=200, l=4, repetitions=100)

Temps moyen pour la Méthode 1 : 0.006501 secondes
Temps moyen pour la Méthode 2 : 0.029518 secondes


Intuitivement, on pourrait penser que le fait de générer beacoup plus de nombre aléatoire par la première méthode prendrait plus de temps, mais du fait que le python est un langage interprété et non compilé, le fait de pouvoir préduir les calculs suivant pour le processeur par l'utilisation d'une méthode compilée permet de gagner beacoup de temps.
#### On utilisera donc la première méthode dans notre programme

### Génération de matrice unimodulaire

Nous allons explorer plusieurs méthodes pour obtenir des matrices unimodulaires. La première étant de partir d'une matrice identiée et d'appliquer des opération élémentaires de sorte à conserver un déterminant de + - 1.
La méthode suivante conciste à prendre deux matrice triangulaires aléatoire de determinant 1 et de faire leur produit.
La dernière est de faire une combinaison des deux méthodes.

In [3]:
import time


def generate_unimodular_matrix1(n, l = 4, U = None, nb_passages = 10):
    
    if U is None:
        U = [[1 if i == j else 0 for j in range(n)] for i in range(n)]
    else :
        if not (U.is_invertible()):
            raise ValueError("Erreur : la matrice générée n'est pas unimodulaire !")
        U = [[int(U[i, j]) for j in range(n)] for i in range(n)]
    
    for _ in range(n * nb_passages): 
        i, j = randint(0, n - 1), randint(0, n - 1)
        
        if i != j:
            coef = randint(-(l*10), l*10 + 1)
            for k in range(n):
                U[j][k] += coef * U[i][k]
        
            if randint(0, 1):
                U[i], U[j] = U[j], U[i]
        
        if randint(0, 1):
            for k in range(n):
                U[i][k] *= -1

    U = Matrix(ZZ, U)

    if not (U.is_invertible()):
        raise ValueError("Erreur : la matrice générée n'est pas unimodulaire !")
    return U

def generate_unimodular_matrix2(n, l = 4, U = None, nb_passages = 10):
    
    if U is None:
        U = identity_matrix(n)
    else :
        if not (U.is_invertible()):
            raise ValueError("Erreur : la matrice générée n'est pas unimodulaire !")
    
    for _ in range(n * nb_passages):  
        i, j = randint(0, n - 1), randint(0, n - 1)
        if i != j:
            coef = randint(-(l*10), l*10 + 1)
            for k in range(n):
                U[j,k] += coef * U[i,k]
        
            if randint(0, 1):
                U.swap_rows(i, j)
        
        if randint(0, 1):
            U[i] *= -1

    if abs(U.determinant()) != 1:
        raise ValueError("Erreur : la matrice générée n'est pas unimodulaire !")
    
    return U

def generate_unimodular_matrix3(n, l = 4):
    
    U1 = MatrixSpace(ZZ, n,n).random_element(x = -(l*10), y = (l*10) + 1 , distribution = 'uniform')
    U2 = MatrixSpace(ZZ, n,n).random_element(x = -(l*10), y = (l*10) + 1 , distribution = 'uniform')
    
    for i in range(n):
        U1[i,i], U2[i,i] = choice([-1,1]),choice([-1,1])
        for j in range(i):
            U1[i,j], U2[i,j] = 0, 0
            
    U = U1 * U2.transpose()
    if not (U.is_invertible()):
        raise ValueError("Erreur : la matrice générée n'est pas unimodulaire !")
    return U

def test_keygen_performance(n, repetitions=10):
    start_time = time.time()
    for _ in range(repetitions):
        generate_unimodular_matrix1(n)
    time_1 = (time.time() - start_time) / repetitions

    start_time = time.time()
    for _ in range(repetitions):
        generate_unimodular_matrix2(n)
    time_2 = (time.time() - start_time) / repetitions
    
    start_time = time.time()
    for _ in range(repetitions):
        generate_unimodular_matrix3(n)
    time_3 = (time.time() - start_time) / repetitions

    print(f"Temps moyen pour generate_unimodular_matrix1 : {time_1:.6f} secondes")
    print(f"Temps moyen pour generate_unimodular_matrix2 : {time_2:.6f} secondes")
    print(f"Temps moyen pour generate_unimodular_matrix3 : {time_3:.6f} secondes")

In [4]:
test_keygen_performance(n=200, repetitions=100)

Temps moyen pour generate_unimodular_matrix1 : 0.674326 secondes
Temps moyen pour generate_unimodular_matrix2 : 0.764545 secondes
Temps moyen pour generate_unimodular_matrix3 : 0.111157 secondes


Au vu des performances pour obtenir un bon aléa, la seconde méthode qui consiste à multiplier deux matrices triangulaire semble plus intéressant. Mais l'aléa de la matrice n'est pas parfait, nous allons donc appliquer quelques transformations élémentaires à la matrice générer par la méthode 2 de sorte à obtenir un aléa plus important. On remarque en effet que dans l'exemple suivant, la dernière ligne de la matrice est toujours beacoup plus "simple" que les autres et se termine systematiquement par un 1 ou -1.

In [5]:
def KeyGen(n, l = 4, debug = False):
    B = MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
    while (B.hermite_form().column(n-1) == vector(ZZ, [0] * n)):
        B = MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
        
    U = generate_unimodular_matrix3(n, 100) # l = 100 pour illustrer la problématique
        
    B_prime = B * U
    
    if(debug):
        print("Matrice B (clé publique) :")
        print(B)
        print("\nMatrice U (unimodulaire) :")
        print(U)
        print("\nLigne problématique :")
        print(U[n-1])
        print("\nMatrice B' (clé privée) :")
        print(B_prime)

    return B, B_prime

In [6]:
Bpr, Bpb = KeyGen(10, debug=1)

Matrice B (clé publique) :
[ 0 -3 -1 -3 -2  4 -4  3 -1 -2]
[-3 -4 -4  1  0  4  3 -1 -3  2]
[ 3  4 -2  4  2  4  0  1 -1  1]
[ 4 -1 -1  0 -1  1  1 -4  3  3]
[ 0  0  4 -1 -4 -4  2  0 -4  0]
[ 4  4  2  1  2  4 -4  2  4 -1]
[ 3 -2 -3  2 -4 -4  3 -3 -4 -2]
[ 1 -3  4 -1  1  1 -4  4 -3  2]
[-3  1 -4  2  2 -4  4  0  0 -2]
[ 1  2 -3  3  1  3  4 -2 -3 -3]

Matrice U (unimodulaire) :
[-1188993 -1107892   475246    98767   -44627   116155  -371116    28348     2441      -62]
[ 1708211   476470 -1961109   354140   548685 -1304207  -501487   139535    11305     -300]
[  -21958    37228 -1389710   252344   227948  -935114  -382677    86489     8333     -199]
[ -405238   217640   396148   228770  -729436  -968155  -755921  -263044   -20611      561]
[  333426  1416484  -272246   247004  -845403 -1475799  -587225  -374517   -29542      800]
[  205772   282675  -532083   293887  -146504  -825599  -311912    -1257      619       -4]
[ -388581   658279   -31760    96384  -462513 -1354057 -1567352  -420933 

On applique donc quelques tranformations élémentaires et on vérifie que cete nouvelle implémentation n'est pas trop consomatrice de ressources.

In [7]:
def KeyGen(n, l = 4, debug = False):
    B = MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
    while (B.hermite_form().column(n-1) == vector(ZZ, [0] * n)):
        B = MatrixSpace(ZZ, n,n).random_element(x = -l, y = l + 1 , distribution = 'uniform')
        
    U = generate_unimodular_matrix1(n, l, generate_unimodular_matrix3(n), 4)
        
    B_prime = B * U
    
    if(debug):
        print("Matrice B (clé publique) :")
        print(B)
        print("\nMatrice U (unimodulaire) :")
        print(U)
        print("\nLigne anciennement problématique :")
        print(U[n-1])
        print("\nMatrice B' (clé privée) :")
        print(B_prime)

    return B, B_prime

In [8]:
Bpr, Bpb = KeyGen(8, debug=1)

Matrice B (clé publique) :
[ 3  1 -3 -3  1 -2 -1  3]
[-2 -4  1 -3 -3 -3 -2 -2]
[ 1 -2 -1 -4  4  2  2  4]
[ 0  4  3 -4 -4 -2  1  2]
[-1 -3 -1 -3  4  3 -2  2]
[-3 -4  1  3  3  0  2 -3]
[ 2 -4  3  2 -3 -1 -2  2]
[-2  4 -2 -2  1  1 -3  3]

Matrice U (unimodulaire) :
[   -359909      36073     616691    2473852   -1592109     606313     224413     -30944]
[ -66433955   32102519   93870687  460125624 -300516812  116263354   44719436   -6132057]
[-576359818 -359967477  730761119  789577028 -689243103  326397644   -8913820   -1775188]
[  27003186    7801068  -39037833 -102564655   70829545  -28640795   -7285947    1081474]
[   9488627   -4587771  -13407028  -65729619   42928771  -16607978   -6388639     876016]
[-255558605  -55204294  232292278  333737919 -315641508  155959404    7509961   -2218407]
[   1433338    1381447   -2341806   -1743707    1499074    -740853     144208     -10254]
[ -26820970   -4713526   36276613  101673076  -70797099   28852451    7568155   -1116257]

Ligne ancienneme

In [9]:
def test_keygen_performance(n, l = 4, repetitions = 10):
    start_time = time.time()
    for _ in range(repetitions):
        generate_unimodular_matrix1(n, l, generate_unimodular_matrix3(n), 4)
    time_1 = (time.time() - start_time) / repetitions
    
    for _ in range(repetitions):
        generate_unimodular_matrix2(n, l, generate_unimodular_matrix3(n), 4)
    time_2 = (time.time() - start_time) / repetitions

    print(f"Temps moyen pour generate_unimodular_matrix1 sur generate_unimodular_matrix3 : {time_1:.6f} secondes")
    print(f"Temps moyen pour generate_unimodular_matrix2 sur generate_unimodular_matrix3 : {time_2:.6f} secondes")

In [10]:
test_keygen_performance(n=200, repetitions=100)

Temps moyen pour generate_unimodular_matrix1 sur generate_unimodular_matrix3 : 0.461369 secondes
Temps moyen pour generate_unimodular_matrix2 sur generate_unimodular_matrix3 : 0.930238 secondes


In [11]:
def generate_unimodular_matrix3(n, l = 4, nb_passages = 10):
    U = identity_matrix(n)
    
    for _ in range(n * nb_passages):  
        i, j = randint(0, n - 1), randint(0, n - 1)
        if i != j:
            coef = randint(-(l*10), l*10 + 1)
            for k in range(n):
                U[j,k] += coef * U[i,k]
        
            if randint(0, 1):
                U.swap_rows(i, j)
        
        if randint(0, 1):
            U[i] *= -1

    if abs(U.determinant()) != 1:
        raise ValueError("Erreur : la matrice générée n'est pas unimodulaire !")
    
    return U
print(generate_unimodular_matrix3(10))

[           -2575847637          -799592976098          1343816956621         22849308930703                -887661           801202834025           -65950961366           150302021651                     12            64663659186]
[       -19594943840558      -6121191730091795      10538126558436238     179182817865845882            -8779217278       6133439220589750       -501590546346679       1178651307228806                   1004        491944055314935]
[     10290805296379027    3215033788542123307   -5537057080153912431  -94148185143284804464          4627320349677   -3221465891813420465     263422651160522905    -619299729655830202                 240316    -258357785560822423]
[      -704681652045761    -220114774349437328     378828375080170267    6441328575729722908          -314736074950     220555223288819591     -18038462451653091      42370585911650386                  78480      17691482751832862]
[       160567336093004      50155340894408561     -86322615000097791   

# TODO Commenter test d'inversion

In [12]:
import time

def test_invertibility_check(n, repetitions=10):
    # Génération d'une grande matrice unimodulaire
    U = generate_unimodular_matrix(n,4)

    # Test avec is_invertible()
    start_time = time.time()
    for _ in range(repetitions):
        U.is_invertible()
    time_invertible = (time.time() - start_time) / repetitions

    # Test avec abs(U.determinant()) != 1
    start_time = time.time()
    for _ in range(repetitions):
        abs(U.determinant()) != 1
    time_determinant = (time.time() - start_time) / repetitions

    print(f"Temps moyen pour .is_invertible(): {time_invertible:.6f} secondes")
    print(f"Temps moyen pour abs(U.determinant()) != 1: {time_determinant:.6f} secondes")

test_invertibility_check(n=200, repetitions=100)

NameError: name 'generate_unimodular_matrix' is not defined

# Comparaison des méthodes de génération exhaustive de vecteurs d'erreurs

In [ ]:
import time
from itertools import product

def test_vector_generation_performance(dim, r, repetitions=10):
    print(f"Test pour dim={dim}, r={r}, répétitions={repetitions}\n")

    # Temps pour la méthode generate_vectors (génération complète)
    full_generation_times = []
    for _ in range(repetitions):
        start_time = time.time()
        vectors = generate_vectors(dim, r)
        full_generation_times.append(time.time() - start_time)

    # Temps pour la méthode generate_vectors_iterative (génération itérative)
    iterative_full_generation_times = []
    iterative_first_generation_times = []
    for _ in range(repetitions):
        # Temps pour obtenir le premier vecteur
        start_time = time.time()
        gen = generate_vectors_iterative(dim, r)
        _ = next(gen, None)
        iterative_first_generation_times.append(time.time() - start_time)

        # Temps total pour parcourir tous les vecteurs
        start_time = time.time()
        gen = generate_vectors_iterative(dim, r)
        for _ in gen:
            pass
        iterative_full_generation_times.append(time.time() - start_time)

    avg_full_generation_time = sum(full_generation_times) / repetitions
    avg_iterative_full_time = sum(iterative_full_generation_times) / repetitions
    avg_iterative_first_time = sum(iterative_first_generation_times) / repetitions
    
    print("generate_vectors :")
    print(f"- Temps moyen (génération complète) : {avg_full_generation_time:.6f} secondes")
    print(f"- Temps moyen (premier vecteur)     : {avg_full_generation_time:.6f} secondes\n")
    
    print("generate_vectors_iterative :")
    print(f"- Temps moyen (génération complète) : {avg_iterative_full_time:.6f} secondes")
    print(f"- Temps moyen (premier vecteur)     : {avg_iterative_first_time:.6f} secondes")

# Fonctions à comparer
def generate_vectors(dim, r):
    all_vectors = [vector(v) for v in product([0, -1, 1], repeat=dim)]
    valid_vectors = [v for v in all_vectors if v.norm() < r]
    return valid_vectors

def generate_vectors_iterative(dim, r):
    for v in product([0, -1, 1], repeat=dim):
        v = vector(v)
        if v.norm() < r:
            yield v

# Test des performances
test_vector_generation_performance(dim=6, r=3, repetitions=10)

Bien que la méthode itérative soit légèrement plus lente pour la génération complète des vecteurs, elle offre un avantage clé dans les attaques par force brute : un accès quasi-instantané au premier résultat (0,000048 s contre 0,167953 s). Cela permet de tester rapidement des solutions sans attendre la génération complète, optimisant ainsi le temps de recherche. De plus, elle réduit la consommation mémoire en générant les vecteurs au fur et à mesure, ce qui est essentiel pour traiter de grands espaces de solutions.